## Header

In [1]:
import os
from time import time, process_time
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf
from JK.utility import *
from JK.LSTM import *

## Data Preprocessing

In [2]:
import JK.DataCleaner

# Demographic FIPS= 3114 , # Motality FIPS= 2939 , # Mobility FIPS= 3172
First date to be trained: 2020-03-01 00:00:00 , Final date to be trained: 2020-05-23 00:00:00
...

KeyboardInterrupt: 

## Model run

In [ ]:
homedir = get_homedir()

TODAY = '0526' # date of today, in the form of string. It signifies which folder in JK/preprocessing to be trained.
date_ed = pd.Timestamp('2020-05-23') # end date to be included in the training. in the form of pandas Timestamp. 
                                     # should be in the same format as the output of DataCleaner.
timedelta = 13

PATH_PREP = f"{homedir}/JK/preprocessing/{TODAY}"
PATH = f"{homedir}/JK/prediction/{TODAY}_test"

split_ratio = None
QUANTILE = list(quantileList)
history_size = 7
target_size = 14
step_size = 1
NUM_CELLS = 128
lr = 0.001
dp = 0.2
EPOCHS = 2
#######################################################################################

with open(PATH_PREP+f'/FIPS.txt', 'r') as f:
    FIPS_total = eval(f.read())

data_ctg = np.load(PATH_PREP+f'/data_ctg.npy', allow_pickle=True)
print(f'Categorical data of shape {data_ctg.shape} is loaded.')
data_ts = np.load(PATH_PREP+f'/data_ts.npy', allow_pickle=True)
print(f'Timeseries data of shape {data_ts.shape} is loaded.')
data_ts = data_ts[:, :-timedelta, :]

with open(PATH_PREP+f'/columns_ctg.txt', 'r') as f:
    columns_ctg = eval(f.read())
with open(PATH_PREP+f'/columns_ts.txt', 'r') as f:
    columns_ts = eval(f.read())
print(f'# of features = {len(columns_ctg)+len(columns_ts)}')

target_idx = columns_ts.index('deaths')
print('target_idx:', target_idx)

try:
    os.mkdir(PATH)
except OSError as error:
    print(error)

X_train, y_train, C_train = train_full(data_ts, data_ctg, target_idx, history_size, target_size, step_size=step_size)

scaler_ts, scaler_ctg = get_StandardScaler(X_train, C_train)

X_train, y_train = normalizer(scaler_ts, X_train, y_train, target_idx)
C_train = normalizer(scaler_ctg, C_train)

train_data = load_Dataset(X_train, C_train, y_train)

for i in range(3):
    model, history = LSTM_fit_mult(train_data, lr=lr, NUM_CELLS=NUM_CELLS, EPOCHS=EPOCHS, dp=dp, monitor=True, earlystop=False, verbose=2)
    FILEPATH = f"/LSTM_mult_hist_size_{history_size}"
    plot_train_history(history, title=f'History size={history_size}, dropout={dp}', path=PATH+FILEPATH+f'_history_{i}.png')
    model.save_weights(PATH+FILEPATH+f'_weights', save_format="tf")

    df_future = predict_future_mult(model, data_ts, data_ctg, scaler_ts, scaler_ctg, history_size, target_idx, FIPS=FIPS_total, date_ed=date_ed-pd.Timedelta(days=timedelta))
    df_future.to_csv(PATH+f'/LSTM_mult_hist_size_{history_size}_{TODAY}_{i}.csv', index=False)

    model_test = SingleLayerConditionalRNN(NUM_CELLS, target_size, dp, quantileList)
    optimizer = tf.keras.optimizers.Adam(learning_rate=lr)
    model_test.compile(optimizer=optimizer, loss=lambda y_p, y: MultiQuantileLoss(quantileList, target_size, y_p, y))
    load_status = model_test.load_weights(PATH+FILEPATH+f'_weights')
    # print(load_status.assert_consumed())
    df_future_test = predict_future_mult(model_test, data_ts, data_ctg, scaler_ts, scaler_ctg, history_size, target_idx, FIPS=FIPS_total, date_ed=date_ed-pd.Timedelta(days=timedelta))
    df_future_test.to_csv(PATH+f'/LSTM_mult_hist_size_{history_size}_{TODAY}_{i}_test.csv', index=False)